In [9]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf


In [10]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [12]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras


train = pd.read_csv("data/train.csv") 
test  = pd.read_csv("data/test.csv")  

train['file_name'] = train['file_name'].apply(lambda x: os.path.join('data', x))
if 'file_name' in test.columns:
    test['file_name'] = test['file_name'].apply(lambda x: os.path.join('data', x))
else:
    # test CSV has column 'id' instead
    test['file_name'] = test['id'].apply(lambda x: os.path.join('data', 'test_data_v2', x))


train_df, val_df = train_test_split(
    train, 
    test_size=0.2, 
    random_state=42, 
    stratify=train['label']
)

def create_generators(train_df, val_df, test_df):
    train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True
    )
    test_val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='file_name',
        y_col='label',
        target_size=(224, 224),
        class_mode='raw',
        batch_size=32,
        shuffle=True
    )
    val_generator = test_val_datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col='file_name',
        y_col='label',
        target_size=(224, 224),
        class_mode='raw',
        batch_size=32,
        shuffle=False
    )
    test_generator = test_val_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='file_name',
        y_col=None,
        target_size=(224, 224),
        class_mode=None,
        batch_size=32,
        shuffle=False
    )

    return train_generator, val_generator, test_generator

train_gen, val_gen, test_gen = create_generators(train_df, val_df, test)

Found 63960 validated image filenames.
Found 15990 validated image filenames.
Found 0 validated image filenames.


/Users/parisbozzuti/Desktop/362final_backup/.venv/lib/python3.11/site-packages/keras/src/legacy/preprocessing/image.py:918: UserWarning: Found 5540 invalid image filename(s) in x_col="file_name". These filename(s) will be ignored.
  warnings.warn(


In [13]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import regularizers
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = True 

for layer in base_model.layers[:-40]:  
    layer.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    verbose=1
)

2025-12-03 14:06:26.329325: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-12-03 14:06:26.329626: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-12-03 14:06:26.329643: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-12-03 14:06:26.329845: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-12-03 14:06:26.329863: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2025-12-03 14:06:28.857392: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1999/1999 ━━━━━━━━━━━━━━━━━━━━ 401s 197ms/step - accuracy: 0.8350 - loss: 0.4128 - val_accuracy: 0.9350 - val_loss: 0.1865
Epoch 2/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 405s 203ms/step - accuracy: 0.9219 - loss: 0.2217 - val_accuracy: 0.9525 - val_loss: 0.1431
Epoch 3/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 418s 209ms/step - accuracy: 0.9416 - loss: 0.1714 - val_accuracy: 0.9623 - val_loss: 0.1223
Epoch 4/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 398s 199ms/step - accuracy: 0.9509 - loss: 0.1484 - val_accuracy: 0.9548 - val_loss: 0.1398
Epoch 5/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 400s 200ms/step - accuracy: 0.9593 - loss: 0.1259 - val_accuracy: 0.9572 - val_loss: 0.1365
Epoch 6/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 396s 198ms/step - accuracy: 0.9640 - loss: 0.1151 - val_accuracy: 0.9717 - val_loss: 0.0992
Epoch 7/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 419s 210ms/step - accuracy: 0.9682 - loss: 0.1050 - val_accuracy: 0.9645 - val_loss: 0.1205
Epoch 8/10
1999/1999 ━━━━━━━━━━━━━━━━━━━━ 414s 207ms/step - accuracy: 0.9

In [14]:
val_loss, val_acc = model.evaluate(val_gen, verbose=0)
print('Initial validation loss', val_loss)
print('Initial validation accuracy', val_acc)



Initial validation loss 0.0950523167848587
Initial validation accuracy 0.9744840264320374
